In [7]:
import os
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import requests
import json
import sseclient
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm

import dotenv

In [8]:
env_file = '.env'
dotenv.load_dotenv(env_file, override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
embeddings_df = pd.read_parquet("data/embeddings_df.parquet", engine='fastparquet')
embeddings_df.head()

,chunk,object,embedding_object,index,embedding,prompt_tokens,total_tokens
level_0,,,,,,,
0,"{'header': ['Alert'], 'link': '/docs/alert', '...",list,embedding,0,"[-0.022364800795912743, 0.031609658151865005, ...",119,119
1,"{'header': ['Alert', 'Appearance'], 'link': '/...",list,embedding,0,"[-0.002292848890647292, 0.04747709631919861, 0...",88,88
2,"{'header': ['Alert', 'Appearance', 'Informatio...",list,embedding,0,"[-0.0034569110721349716, 0.047789670526981354,...",60,60
3,"{'header': ['Alert', 'Appearance', 'Positive']...",list,embedding,0,"[0.004593235906213522, 0.036937415599823, 0.00...",40,40
4,"{'header': ['Alert', 'Appearance', 'Warning'],...",list,embedding,0,"[-0.002690419787541032, 0.03875597566366196, 0...",53,53


In [64]:
try:
    # Read `embeddings_cache` from parquet file
    embeddings_cache = pd.read_parquet("data/embeddings_cache.parquet", engine='fastparquet')
except:
    # If file does not exist, create an empty dataframe
    embeddings_cache = pd.DataFrame(columns=['embedding'])
    # Write to parquet file
    embeddings_cache.to_parquet("data/embeddings_cache.parquet", engine='fastparquet')

embeddings_cache
    

,embedding


In [65]:
# Add a new row to embeddings_cache
# Key: str(embeddings_df.loc[0, "chunk"])
# Value: embeddings_df.loc[0, "embedding"]
# Column: "embedding"

embeddings_cache.loc[str(embeddings_df.loc[0, "chunk"]), "embedding"] = embeddings_df.loc[0, "embedding"]
embeddings_cache.loc[str(embeddings_df.loc[1, "chunk"]), "embedding"] = embeddings_df.loc[1, "embedding"]
embeddings_cache.loc["What is a trapezoid?", "embedding"] = embeddings_df.loc[2, "embedding"]
embeddings_cache

,embedding
"{'header': ['Alert'], 'link': '/docs/alert', 'content': ""A small box to quickly grab the user's attention and communicate a brief message. An alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime.""}","[-0.022364800795912743, 0.031609658151865005, ..."
"{'header': ['Alert', 'Appearance'], 'link': '/docs/alert#appearance', 'content': 'The alert message must be concise, we recommend no more than 256 characters. If you feel the need to explain in further detail, you may use an optional hyperlink at the end of the alert message to explain more on a new page. You may also include an optional close icon (close icon is required for sticky alerts).'}","[-0.002292848890647292, 0.04747709631919861, 0..."
What is a trapezoid?,"[-0.0034569110721349716, 0.047789670526981354,..."


In [72]:
embeddings_cache.index

Index(['{'header': ['Alert'], 'link': '/docs/alert', 'content': "A small box to quickly grab the user's attention and communicate a brief message. An alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."}',
       '{'header': ['Alert', 'Appearance'], 'link': '/docs/alert#appearance', 'content': 'The alert message must be concise, we recommend no more than 256 characters. If you feel the need to explain in further detail, you may use an optional hyperlink at the end of the alert message to explain more on a new page. You may also include an optional close icon (close icon is required for sticky alerts).'}

In [66]:
str(embeddings_df.loc[0, "chunk"]) in embeddings_cache.index

True

In [67]:
embeddings_cache.loc[str(embeddings_df.loc[0, "chunk"]), "embedding"]

[-0.022364800795912743,
 0.031609658151865005,
 0.007148155942559242,
 6.914396362844855e-05,
 0.01847587153315544,
 -0.005594660062342882,
 -0.009473209269344807,
 -0.016081620007753372,
 -0.014081797562539577,
 -0.009141058661043644,
 -0.008324522525072098,
 0.008103088475763798,
 -0.03404542803764343,
 -0.0035308292135596275,
 -0.008940384723246098,
 0.006058286875486374,
 0.018738824874162674,
 -0.0210777185857296,
 0.0010059663327410817,
 -0.010172109119594097,
 -0.011092443019151688,
 0.010594217106699944,
 0.0026675837580114603,
 0.019998228177428246,
 -0.01188822090625763,
 0.0039027684833854437,
 0.017617816105484962,
 -0.028952453285455704,
 -0.0032505772542208433,
 -0.02480057254433632,
 0.02803904004395008,
 0.009583926759660244,
 -0.011798263527452946,
 -0.013839605264365673,
 -0.0017031363677233458,
 0.01301614847034216,
 0.01487757544964552,
 -0.00835220143198967,
 0.03808659315109253,
 -0.01299538929015398,
 -0.001076894230209291,
 0.0012429695343598723,
 0.013355218805

In [92]:
def get_and_cache_embedding(query, model='text-embedding-ada-002'):
    if query in embeddings_cache.index:
        print(1)
        return embeddings_cache.loc[query, "embedding"]
    else:
        print(2)
        embeddings_cache.loc[query, "embedding"] = get_embedding(query, model=model)
        embeddings_cache.to_parquet("data/embeddings_cache.parquet", engine='fastparquet')
        return embeddings_cache.loc[query, "embedding"]

In [98]:
print(get_and_cache_embedding("What is a trapezoid?"))
print(get_and_cache_embedding("What is a triangle?"))
print(get_and_cache_embedding("What is a rectangle?"))

1
[-0.0034569110721349716, 0.047789670526981354, 0.010372468270361423, 0.004357130266726017, 0.018025198951363564, 0.0016928977565839887, -0.016165785491466522, -0.01080956868827343, -0.006618952844291925, -0.021549755707383156, -0.0020519448444247246, 0.013945592567324638, -0.022909624502062798, 0.01148950308561325, -0.012259632349014282, 0.007846997119486332, 0.027516527101397514, -0.029389815405011177, 0.010948330163955688, 0.0011274421121925116, -0.00436059944331646, 0.02293737791478634, -0.009914552792906761, 0.010837321169674397, -0.02129998430609703, 0.007673544809222221, 0.006723024416714907, -0.03027789294719696, -0.002201114082708955, -0.021091841161251068, 0.032997630536556244, 0.013508491218090057, -0.0012523280456662178, -0.026683954522013664, 0.003850648645311594, 0.004135110881179571, 0.003120413050055504, -0.01631842367351055, 0.03560635820031166, -0.0229790061712265, -0.02060617320239544, 0.004815045278519392, 0.01298119593411684, -0.023506302386522293, -0.004339784849

RetryError: RetryError[<Future at 0x163bb737be0 state=finished raised InvalidRequestError>]

In [ ]:
def get_similarities(df, query, pprint=True):
  res = df.copy()

  embedding = get_and_cache_embedding(query)
  res['similarities'] = res["embedding"].apply(lambda x: cosine_similarity(x, embedding))
  res = res.sort_values('similarities', ascending=False)
  return res

res = get_similarities(embeddings_df, 'What are the different variants of Button?')
res.head()